卷积神经网络是深度学习中最经典的模型之一。它巧妙地利用很少的权重却达到了全连接网络实现不了的效果。
# 全连接网络的局限性
在mnist多层分类中，仅使用一个28x28像素的小图片数据集就完成了分类任务。但在实际应用中要处理的图片像素一般都是1024，甚至更大。如果只有两个隐藏层，每层各用了256个节点，则MNIST数据集所需要的参数是(28x28x256+256x256+256x10)个w,再加上(256+ 256 +10)个b。

1. 图像变大导致色彩数变多，不好解决
如果换为1000像素，仅一层就需要1000x1000x256$ \approx $2亿个W。这只是灰度图，如果是RGB的真彩色图，再乘以3约等于6亿。如果再加上更多层，需要的学习参数将是非常多的，不仅消耗大量的内存，同时也需要大量的运算。

2. 不便处理高维数据
对于比较复杂的高维数据，如果按照全连接的方法，则只能通过增加节点、增加层数的方式来解决。而增加节点会引起参数过多的问题。因为由于隐藏层神经网络使用的是Sigmoid或Tanh激活函数，其反向传播的有效层数也只能在4~6层左右。所以，层数再多只会使反向传播的修正值越来越小，网络无法训练。

而卷积网络使用了**参数共享的方式**，换了一个角度来解决问题，不仅在准确率上大大提升，也把参数维度降了下来。

# 理解卷积神经网络
卷积网络避免了对参数的过度依赖，相比全连接神经网络，能更好地识别高维数据。
![sobel算子](imgs/17_sobel.png)上图为sobel算子对图片处理后的效果，它可以把图片的轮廓显示出来。它其实是一个很简单的矩阵计算。![](imgs/17_matrix_cal.png)图中5x5矩阵可以理解为图（小女孩原图），经过卷及操作后，变为轮廓图。
整个过程步骤如下：
1. 在外面补了一圈0，这个过程叫做padding，目的是为了变换后生成同样大小的矩阵。
2. 将3x3的矩阵中的每个元素分别与3x3矩阵对应位置上的元素相乘，然后再相加，这样得到的值作为图b的第一个元素。
3. 中间的3x3矩阵就是sobel算子。
4. 把图a中左上角的3x3矩阵向右移动一个格子，这可以理解为步长为1.
5. 将图a矩阵中的每个元素分别与中间的3x3矩阵对应位置上的元素相乘然后进行加和运算，算出的值填到图b的第二个元素里。
6. 一直重复上述操作，直到将图b中的值都填满，整个过程就叫做卷积。 

sobel可以理解为卷积神经网络里的卷积核（也可以叫做“滤波器”，filter），它里面的值也可以理解为权重w。在sobel中，这些w是固定的，就相当于一个训练好的模型，只要通过里面的值变换后的图片，就会产生具有轮廓的效果。这个变换后的图片，在卷积神经网络里称为**feature map**。

> 新生成的图片里面的每个像素值并不能保证在0~256之间。对于在区间外的像素点会导致灰度图无法显示，所以还需要做一次归一化，然后每个值都乘以256，再将所有的值映射到这个区间内。归一算法为：x=(x-Min) / (Max - Min)。其中，Max与Min为整体数据里的最大值和最小值，x是当前要转换的像素值。归一化之后可以保证每个x都在[0,1]的区间内。

# 网络结构
卷积神经网络的结构与全连接网络相比复杂很多。它的网络结构主要包括**卷积层、池化层**。细节可以分为**滤波器、步长、卷及操作、池化操作**。
## 网络结构描述
前面讲述的是一个基本原理，实际的卷积操作会复杂一些，对于一幅图片一般会使用多个卷积核（滤波器），将它们统一放到卷积层里来操作，**这一层中有几个滤波器，就会得到几个feature map**，接着还要经理一个池化层（pooling），将生成的feature map缩小（降维）。下图为一个神经网络中一个标准的卷积操作组合。![](imgs/17_convolution_network.png) 图中卷积层里面有channel的个数代表卷积层的深度。池化层中则只有一个滤波器(fileter)，主要参数是尺寸大小(即步长大小)。一个卷积网络的完整结构，如图：![](imgs/17_convolution_fully_network.png) 
一个卷积神经网络里面包括**5部分**--**输入层、若干个卷积操作和池化层结合的部分、全局平均池化层、输出层**：
- 输入层：将每个像素代表一个特征节点输入进来。
- 卷积操作部分：由多个滤波器组合的卷积层。
- 池化层：将卷积结果降维。
- 全局平均池化层：对省城的feature map取全局平均值。
- 输出层：需要分成几类，相应的就会有几个输出节点。每个输出节点都代表当前样本属于的该类型的概率。

> 全局平均池化层是后出的技术，在以前的教材里，这个位置通过是使用1~3个全连接层来代替。全连接层的劣势在于会产生大量的计算，需要大量的参数，但在效果上却和全局平均池化层一样。所以，直接使用效率更高的全局平均池化层。

# 卷积操作
卷积分为**窄卷积、全卷积和同卷积**。

1. 步长
步长是卷积操作的核心。通过步长的变换，可以得到想要的不同类型的卷积操作。![卷积](imgs/17_Narrow convolution.png)图中5x5大小的矩阵代表图片，每个图片右侧的3x3矩阵代表卷积核，最右侧的3x3矩阵为计算完的结果feature map。

卷积操作仍然是将卷积核（filter）对应的图片（image）中的矩阵数据--相乘，再相加。第一行feature map中的第一个元素，是由image块中前3行3列中的每个元素与filter中对应元素相乘再相加得到的(4=1x1+1x0+1x1+0x0+1x1+1x0+0x1+0x0+1)

步长（stride）表示卷积核在图片上移动的格数。
- 当步长为1情况下，第二行右边的feature map块里的第二个元素3，是由卷积核计算完第一个元素4，右移一格后计算得来的，相当于图片中的前3行和第1到第4列围成的3x3矩阵与卷积核各对应元素进行相乘相加操作。
- 当步长为2的情况下，就代表每次移动2格，最终会得到一个图中第二行左边的2x2矩阵块的结果。


2. 窄卷积
窄卷积(valid卷积)，即生成的feature map比原来的原始图片小，它的步长是可变的。假如滑动步长为S，原始图片的维度为N1xN1，那么卷积核的大小为N2xN2，卷积后的图像大小(N1-N2)/S+1x(N1-N2)/S+1。

3. 同卷积
同卷积(same卷积)，代表的意思是卷积后的图片尺寸与原始图片尺寸一样大，同卷积步长是固定的，滑动步长为1.一般操作时都要使用padding技术（外围补一圈0，以确保生成的尺寸不变）。

4. 全卷积
全卷积（full卷积），也叫反卷积，就是把原始图片里的每个像素点都用卷积操作展开。![](imgs/17_full_convolution.png)
图中，白色的块是原始图片，浅色的是卷积核，深色的是正在卷积操作的像素点。反卷积操作的过程中，同样需要对原有图片进行padding操作，生成的结果会比原有的图片尺寸大。 全卷积的步长也是固定的，滑动步长为1，加入原始图片的维度为N1xN1,那么卷积核的大小为N2xN2，卷积后的图像大小，即N1+N2xN1+N2-1

窄卷积和同卷积都是卷积网络里常用的技术，然而全卷积（full卷积）却相反，它更多地用在反卷积网络中。

5. 反向传播
反向传播的核心步骤主要有两步：
- 反向将误差传到前面一层。
- 根据当前的误差对应的学习参数表达式，计算出其需要更新的差值。

反向求导，仍然使用链式求导法则，找到是误差最小化的梯度，再配合学习率算出更新的差值。将生成的feature map做一次padding后，与转置后的卷积核做一次卷积操作即可得到输入端的误差，从而实现误差的反向传递。

6. 多通道卷积

通道（Channel），是指图片中每个像素由几个数来表示，这几个数一般指的是色彩。比如一个灰度图的通道就是1，一个彩色图的通道就是3（红黄蓝）。

在卷积神经网络里，通道又分为**输入通道**和**输出通道**。
- 输入通道：就是图片的通道。如果是彩色图片，起始的输入通道就是3.如果是中间层的卷积，输入通道就是上一层的输出通道个数，计算方法是，每个输入通道的图片都使用同一个卷积核进行卷积操作，生成与输入通道匹配的feature map（比如彩色图片就是3个），然后再把这几张feature map相同位置上的值加起来，生成一张feature map。

- 输出通道：想要输出ji